In [ ]:
#####################Вся эта хрень собирает из наших датасетов посты (сделанные именно нашими организациями) и комменты к ним#####################
#https://vk.com/dev/wall.get - используется для получения постов
#https://vk.com/dev/wall.getComments - используется для получения комментариев

import json
import urllib # Чтобы без проблем слать запросики к API
from urllib.request import urlopen 
import mysql_credits # Доступ к БД
import pymysql
from pprint import pprint # Красиво выводим JSON
from time import sleep # Даём серверу немного поспать, чтобы не переутомлялся (чтобы не банили за превышение частоты запросов)
import datetime # Чтобы конвертировать unix-time stamp в обычный формат даты для БД
from os import listdir # Для поиска по директории
from os.path import isfile, join # Для проверки, чем является элемент: файлом или директорией
import hashlib # Чтобы создавать уникальные id-шники для постов и чтобы исключить повторения
import re # Для экранирования текстовой информации

# Объявляем всякие очевидные переменные
vk_api_group_method = 'https://api.vk.com/method/wall.get?v=5.52'
vk_api_comment_method = 'https://api.vk.com/method/wall.getComments?v=5.52'
vk_token = '&access_token=d45895518d458f512d2104984d776663f6e3556a88dd87e90df75aa4ea7668338fcb3bf4284b7294bf5d5'
vk_group_param = '&owner_id=-'
vk_post_param = '&post_id='

# Коннектимся к божественному, бесплатному MySQL
connection = pymysql.connect(
    host=mysql_credits.db_host,
    user=mysql_credits.db_user,
    password=mysql_credits.db_password,
    db=mysql_credits.db,
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

#Экранирование символов
def do_escaping(string_to_make_escaped):
    return string_to_make_escaped.translate(str.maketrans({"-":  r"\-",
                                          "]":  r"\]",
                                          "\\": r"\\",
                                          "^":  r"\^",
                                          "$":  r"\$",
                                          "*":  r"\*",
                                          "'":  r"\'",
                                          '"':  r'\"',
                                          ".":  r"\."}))

# Начинаем творить
path_with_jsons = 'cultural_data'
json_files = [f for f in listdir(path_with_jsons) if isfile(join(path_with_jsons, f))]

for json_file in json_files:
    with connection.cursor() as cursor:
        with open('cultural_data/'+json_file+'', 'rt') as data_file:
            theaters = json.load(data_file)
            for theater in theaters:
                # Проверяем, имеются вообще соцсети у организации
                if 'socialGroups' in theater['data']['general']['organization']:
                    for social_group in theater['data']['general']['organization']['socialGroups']:
                        if social_group['network'] =='vk':
                            # Проверяем на наличие в базе организации, если нет, то добавляем
                            check_org_sql = "SELECT organisation.organisation_vk_id FROM organisation WHERE organisation.organisation_id = '"+theater['_id']+"'"
                            cursor.execute(check_org_sql)
                            existing_organisations = cursor.fetchall()
                            connection.commit()
                            if len(existing_organisations) == 0:
                                new_org_sql = "INSERT INTO organisation (organisation_id, organisation_vk_id) VALUES ('"+theater['_id']+"', '"+social_group['networkId']+"')"
                                cursor.execute(new_org_sql)
                                connection.commit()
                            sleep(0.5) # Спим полсекунды, чтобы не банили
                            group_posts_web_url = urllib.request.urlopen(
                                vk_api_group_method+
                                vk_token+
                                vk_group_param+social_group['networkId']
                            )
                            print('Запрос на посты из группы вк: '+group_posts_web_url.url)
                            group_posts_data = group_posts_web_url.read()
                            encoding = group_posts_web_url.info().get_content_charset('utf-8')
                            vk_group_data_json = json.loads(group_posts_data.decode(encoding))
                            #pprint(vk_group_data_json)
                            if 'response' in vk_group_data_json:
                                for vk_group_item in vk_group_data_json['response']['items']:
                                    if vk_group_item['from_id'] == vk_group_item['owner_id']:
                                        owner_or_not = 'owner'
                                    else:
                                        owner_or_not = 'not_owner'
                                    print(owner_or_not)
                                    # Проверяем, имеется ли вообще текст у поста.
                                    if 'text' in vk_group_item:
                                        # Проверяем на наличие в базе поста, если нет, то добавляем
                                        print(vk_group_item['id'])
                                        escaped_text = do_escaping(vk_group_item['text'])
                                        article_text_hash = hashlib.md5(escaped_text.encode('utf-8')).hexdigest()
                                        check_article_sql = "SELECT article.article_hash FROM article WHERE article.article_hash = '"+str(article_text_hash)+"' "
                                        cursor.execute(check_article_sql)
                                        existing_articles = cursor.fetchall()
                                        connection.commit()
                                        if len(existing_articles) == 0:
                                            new_article_sql = "INSERT INTO article (article_hash, article_id, article_origin, article_belonging, article_text, article_date) VALUES ('"+article_text_hash+"', '"+str(vk_group_item['id'])+"', 'vk', '"+owner_or_not+"', '"+escaped_text+"', '"+datetime.datetime.fromtimestamp(int(vk_group_item['date'])).strftime('%Y-%m-%d %H:%M:%S')+"')"
                                            try:
                                                cursor.execute('SET NAMES utf8mb4')
                                                cursor.execute(new_article_sql)
                                                cursor.execute("INSERT INTO article4organisation (article_hash, organisation_id) VALUES ('"+article_text_hash+"', '"+theater['_id']+"')")
                                                connection.commit()
                                                print('Всё збс, в бд добавлено!')
                                            except:
                                                print('Ошибка возникла в этом запросе:')
                                                print(new_article_sql)
                                        sleep(0.3) # Спим треть секунды, чтобы не банили
                                        # Тут уже берём комменты от поста
                                        vk_comments_web_url = urllib.request.urlopen(
                                            vk_api_comment_method+
                                            vk_token+
                                            vk_group_param.replace('-', '')+str(vk_group_item['owner_id'])+
                                            vk_post_param+str(vk_group_item['id'])
                                        )
                                        post_comments_data = vk_comments_web_url.read()
                                        encoding = vk_comments_web_url.info().get_content_charset('utf-8')
                                        post_comments_json = json.loads(post_comments_data.decode(encoding))
                                        if 'response' in post_comments_json:
                                            if 'items' in post_comments_json['response']:
                                                for comment in post_comments_json['response']['items']:
                                                    escaped_comment = do_escaping(comment['text'])
                                                    article_comment_hash = hashlib.md5(escaped_comment.encode('utf-8')).hexdigest()
                                                    check_article__comment_sql = "SELECT comment.comment_hash FROM comment WHERE comment.comment_hash = '"+str(article_comment_hash)+"' "
                                                    cursor.execute(check_article__comment_sql)
                                                    existing_comments = cursor.fetchall()
                                                    connection.commit()
                                                    if len(existing_comments) == 0:
                                                        new_comment_sql = "INSERT INTO comment (comment_hash, comment_id, comment_from_id, comment_text, comment_date) VALUES ('"+article_comment_hash+"', '"+str(comment['id'])+"', '"+str(comment['from_id'])+"', '"+escaped_comment+"', '"+datetime.datetime.fromtimestamp(int(comment['date'])).strftime('%Y-%m-%d %H:%M:%S')+"')"
                                                        try:
                                                            cursor.execute('SET NAMES utf8mb4')
                                                            cursor.execute(new_comment_sql)
                                                            cursor.execute("INSERT INTO comment4article (article_hash, comment_hash) VALUES ('"+article_text_hash+"', '"+article_comment_hash+"')")
                                                            connection.commit()
                                                            print('КОММЕНТ: Всё збс, в бд добавлено!')
                                                        except:
                                                            print('КОММЕНТ: Ошибка возникла в этом запросе:')
                                                            print(new_comment_sql)
                                        else:
                                            print('ОШИБКА БЛЯЯЯТЬ! Ответ на запрос комментов какой-то хреновый!')
                                            pprint(post_comments_json)
                                        print('\n')

                            else:
                                print('ОШИБКА API БЛЯЯЯТЬ!')
                                print(vk_group_data_json)
                            #print(json.dumps(json_data, indent=4,  ensure_ascii=False).encode('utf8').decode('utf8'))
                else:
                    print('Нет соц. сетей!')
            

Нет соц. сетей!
Нет соц. сетей!
Нет соц. сетей!
Запрос на посты из группы вк: https://api.vk.com/method/wall.get?v=5.52&access_token=d45895518d458f512d2104984d776663f6e3556a88dd87e90df75aa4ea7668338fcb3bf4284b7294bf5d5&owner_id=-25684989
not_owner
100


owner
97


owner
95


not_owner
91


owner
90


not_owner
89


owner
88


not_owner
86
Всё збс, в бд добавлено!


not_owner
85
Всё збс, в бд добавлено!


not_owner
84


not_owner
83
Всё збс, в бд добавлено!


not_owner
79
Всё збс, в бд добавлено!


not_owner
78
Всё збс, в бд добавлено!


owner
77
Всё збс, в бд добавлено!


owner
76
Всё збс, в бд добавлено!


owner
74
Всё збс, в бд добавлено!


not_owner
73
Всё збс, в бд добавлено!


not_owner
68
Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!


not_owner
67
Всё збс, в бд добавлено!


not_owner
66
Всё збс, в бд добавлено!


Запрос на посты из группы вк: https://api.vk.com/method/wall.get?v=5.52&access_token=d4



owner
1785


owner
1784


owner
1783


owner
1782


owner
1781


owner
1780


owner
1779


owner
1778


owner
1777


owner
1772


not_owner
1762


owner
1761


owner
1760


not_owner
1757


owner
1756


owner
1755


Нет соц. сетей!
Запрос на посты из группы вк: https://api.vk.com/method/wall.get?v=5.52&access_token=d45895518d458f512d2104984d776663f6e3556a88dd87e90df75aa4ea7668338fcb3bf4284b7294bf5d5&owner_id=-45958490
owner
69
Всё збс, в бд добавлено!


owner
68
Всё збс, в бд добавлено!


owner
67
Всё збс, в бд добавлено!


owner
65
Всё збс, в бд добавлено!


owner
62
КОММЕНТ: Всё збс, в бд добавлено!


owner
61


owner
58


owner
56


owner
55


owner
54


owner
53
Всё збс, в бд добавлено!


owner
51
Всё збс, в бд добавлено!


owner
49
Всё збс, в бд добавлено!


owner
47


owner
46
Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!


owner
45


owner
42
Всё збс, в бд добавлено!


owner
41
Всё збс, в бд добав



owner
11
Всё збс, в бд добавлено!


owner
10
Всё збс, в бд добавлено!


owner
9
Всё збс, в бд добавлено!


owner
8


owner
7
Всё збс, в бд добавлено!


owner
6
Всё збс, в бд добавлено!


owner
5
Всё збс, в бд добавлено!


owner
4
Всё збс, в бд добавлено!


owner
3
Всё збс, в бд добавлено!


owner
2
Всё збс, в бд добавлено!


owner
1


Нет соц. сетей!
Нет соц. сетей!
Нет соц. сетей!
Нет соц. сетей!
Запрос на посты из группы вк: https://api.vk.com/method/wall.get?v=5.52&access_token=d45895518d458f512d2104984d776663f6e3556a88dd87e90df75aa4ea7668338fcb3bf4284b7294bf5d5&owner_id=-26575799
Нет соц. сетей!
Запрос на посты из группы вк: https://api.vk.com/method/wall.get?v=5.52&access_token=d45895518d458f512d2104984d776663f6e3556a88dd87e90df75aa4ea7668338fcb3bf4284b7294bf5d5&owner_id=-47114993
owner
1949
Всё збс, в бд добавлено!


owner
1947
Всё збс, в бд добавлено!


not_owner
1946
Всё збс, в бд добавлено!


owner
1944
Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё 



owner
377


not_owner
376


owner
373


owner
372


owner
371


owner
370


not_owner
369


owner
368


not_owner
367


owner
366


owner
365


owner
364


owner
363


owner
362


owner
361


owner
360


owner
359


owner
358


owner
357


Запрос на посты из группы вк: https://api.vk.com/method/wall.get?v=5.52&access_token=d45895518d458f512d2104984d776663f6e3556a88dd87e90df75aa4ea7668338fcb3bf4284b7294bf5d5&owner_id=-38532767
owner
6027


owner
6105


owner
6104


owner
6102


owner
6101


owner
6100


owner
6098


owner
6097


owner
6094


owner
6093


owner
6092


owner
6091


owner
6090


owner
6089


owner
6088


owner
6087


owner
6085


owner
6084


owner
6083


not_owner
6077


Запрос на посты из группы вк: https://api.vk.com/method/wall.get?v=5.52&access_token=d45895518d458f512d2104984d776663f6e3556a88dd87e90df75aa4ea7668338fcb3bf4284b7294bf5d5&owner_id=-102544520
owner
378


owner
377


not_owner
376


owner
373


owner
372


owner
371


owner
370


not_owner
369


owner
368

КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!


owner
1064
Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!


owner
1053
Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!


owner
1041
Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!


owner
1023
Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено



owner
1423


owner
1396
Всё збс, в бд добавлено!


owner
1393
Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!


owner
1392


owner
1384
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!


owner
1381


owner
1380


owner
1378
Всё збс, в бд добавлено!


owner
1373


owner
1361
Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!


owner
1358
Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!


owner
1357
Всё збс, в бд добавлено!


owner
1350


owner
1349
Всё збс, в бд добавлен



owner
354


owner
353


owner
352


owner
351


owner
350


owner
349


owner
348


owner
347


owner
346


owner
345


not_owner
344
Всё збс, в бд добавлено!


not_owner
343
Всё збс, в бд добавлено!


owner
342


owner
341


owner
340


owner
339


owner
338


owner
337


owner
336


Запрос на посты из группы вк: https://api.vk.com/method/wall.get?v=5.52&access_token=d45895518d458f512d2104984d776663f6e3556a88dd87e90df75aa4ea7668338fcb3bf4284b7294bf5d5&owner_id=-122537604
owner
41


not_owner
40
Всё збс, в бд добавлено!


owner
39


not_owner
38
Всё збс, в бд добавлено!


owner
36


not_owner
35
Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!


owner
34


owner
33


not_owner
32
Всё збс, в бд добавлено!


not_owner
31


not_owner
30
Всё збс, в бд добавлено!


not_owner
29


not_owner
28
Всё збс, в бд добавлено!


not_owner
25


not_owner
24


not_owner
23


not_owner
22


not_owner
21
Всё збс, в бд добавлено!


not_owner
20
Всё збс, в бд добавлено!


not_owner
19
Всё збс, 

Запрос на посты из группы вк: https://api.vk.com/method/wall.get?v=5.52&access_token=d45895518d458f512d2104984d776663f6e3556a88dd87e90df75aa4ea7668338fcb3bf4284b7294bf5d5&owner_id=-274265896
ОШИБКА API БЛЯЯЯТЬ!
{'error': {'error_code': 15, 'error_msg': 'Access denied: wall is disabled', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'v', 'value': '5.52'}, {'key': 'owner_id', 'value': '-274265896'}]}}
Запрос на посты из группы вк: https://api.vk.com/method/wall.get?v=5.52&access_token=d45895518d458f512d2104984d776663f6e3556a88dd87e90df75aa4ea7668338fcb3bf4284b7294bf5d5&owner_id=-107734004
owner
1527
Всё збс, в бд добавлено!


owner
1531
Всё збс, в бд добавлено!


owner
1530
Всё збс, в бд добавлено!


owner
1529
Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!


owner
1525
Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!
КОММЕНТ: Всё збс, в бд добавлено!


owner
1524
Всё збс, в бд до